# 03 — Equity & Trades Plot
Loads `data/pnl_demo_equity.csv` and `data/pnl_demo_trades.csv` and plots equity curve, drawdowns, and trade markers.

In [ ]:

from pathlib import Path
import sys

PROJECT_OVERRIDE = r"C:\Users\speed\Desktop\Forex CFD's system"
ROOT = Path(PROJECT_OVERRIDE)
DATA_1D = ROOT / "data" / "prices_1d"
DATA_1H = ROOT / "data" / "prices_1h"
CFG = ROOT / "config" / "baseline.yaml"
DATA_DIR = DATA_1D if DATA_1D.exists() else DATA_1H

print("Project root:", ROOT)
print("Using data dir:", DATA_DIR)
assert DATA_DIR.exists(), f"Expected data under {ROOT/'data'} (prices_1d or prices_1h)"
assert CFG.exists(), f"Missing baseline config at {CFG}"


In [ ]:
import pandas as pd, plotly.graph_objects as go
eq_path = ROOT/'data'/'pnl_demo_equity.csv'
tr_path = ROOT/'data'/'pnl_demo_trades.csv'
eq = pd.read_csv(eq_path, parse_dates=['ts']).set_index('ts')
fig = go.Figure()
fig.add_trace(go.Scatter(x=eq.index, y=eq['portfolio_equity'], name='Portfolio'))
for c in eq.columns:
    if c.startswith('equity_'):
        fig.add_trace(go.Scatter(x=eq.index, y=eq[c], name=c))
fig.update_layout(title='Equity Curves', xaxis_title='UTC', yaxis_title='Equity (base=1.0)')
fig.show()

dd = eq['portfolio_equity']/eq['portfolio_equity'].cummax()-1.0
figdd = go.Figure()
figdd.add_trace(go.Scatter(x=dd.index, y=dd.values, name='Drawdown'))
figdd.update_layout(title='Portfolio Drawdown', xaxis_title='UTC', yaxis_title='Drawdown')
figdd.show()

if tr_path.exists():
    tr = pd.read_csv(tr_path, parse_dates=['entry_time','exit_time'])
    # Mark trade exits on portfolio equity
    exits = tr[['exit_time','symbol','side']].copy()
    exits['y'] = eq['portfolio_equity'].reindex(exits['exit_time']).values
    fig2 = go.Figure(fig)
    fig2.add_trace(go.Scatter(x=exits['exit_time'], y=exits['y'], mode='markers',
                              marker=dict(size=6), name='Trade exits'))
    fig2.update_layout(title='Equity with Trade Exit Markers')
    fig2.show()


In [ ]:
import pandas as pd, plotly.express as px
eq_monthly = (1+eq['portfolio_equity'].pct_change().fillna(0)).resample('M').prod()-1
heat = eq_monthly.to_frame('ret').reset_index()
heat['Year']=heat['ts'].dt.year; heat['Month']=heat['ts'].dt.month_name().str[:3]
pivot = heat.pivot(index='Year', columns='Month', values='ret').fillna(0)
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
pivot = pivot.reindex(columns=[m for m in months if m in pivot.columns])
px.imshow(pivot, text_auto='.1%', aspect='auto', origin='lower', labels=dict(color='Return'),
          title='Monthly Return Heatmap').show()
